In [65]:
%reload_ext autoreload
%autoreload 2
from pywinusb import hid
from lib.DMDpycrafter import DMDhid

In [71]:
dmd = DMDhid()

IndexError: list index out of range

In [75]:
vendor_num = 0x0451
#usb Product ID
product_num = 0xc900

filter = hid.HidDeviceFilter(vendor_id = vendor_num, product_id = product_num)
devices = filter.get_devices()

In [76]:
devices

[]

In [ ]:
def checkforerrors(self):
    self.command('r',0x22,0x01,0x00,[])
    if self.ans[6]!=0:
        print (self.ans[6])  

In [57]:
convlen(len([0xff,0x01,0xff,0x01])+2,16)

'0000000000000110'

In [61]:
int('00010011',2)

19

In [58]:
def bitstobytes(a):
    bytelist=[]
    if len(a)%8!=0:
        padding=8-len(a)%8
        a='0'*padding+a
    for i in range(len(a)//8):
        bytelist.append(int(a[8*i:8*(i+1)],2))

    bytelist.reverse()

    return bytelist

def convlen(a,l):
    b=bin(a)[2:]
    padding=l-len(b)
    b='0'*padding+b

    return b

bitstobytes(convlen(len([0xff,0x01,0xff,0x01])+2,16))

[6, 0, 0, 0]

In [ ]:
def command(self,mode,sequencebyte,com1,com2,data=None):
    buffer = []

    flagstring=''
    if mode=='r':
        flagstring+='1'
    else:
        flagstring+='0'        
    flagstring+='1000000'
    buffer.append(bitstobytes(flagstring)[0])
    buffer.append(sequencebyte)
    temp=bitstobytes(convlen(len(data)+2,16))
    buffer.append(temp[0])
    buffer.append(temp[1])
    buffer.append(com2)
    buffer.append(com1)

    if len(buffer)+len(data)<65:
    
        for i in range(len(data)):
            buffer.append(data[i])

        for i in range(64-len(buffer)):
            buffer.append(0x00)


        self.dev.write(1, buffer)

    else:
        for i in range(64-len(buffer)):
            buffer.append(data[i])

        self.dev.write(1, buffer)

        buffer = []

        j=0
        while j<len(data)-58:
            buffer.append(data[j+58])
            j=j+1
            if j%64==0:
                self.dev.write(1, buffer)

                buffer = []

        if j%64!=0:

            while j%64!=0:
                buffer.append(0x00)
                j=j+1


            self.dev.write(1, buffer)                
            





    self.ans=self.dev.read(0x81,64)

In [19]:
bitstobytes('111')

[7]

In [12]:
convlen(5, 2)

'101'

In [1]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

from lib.utils import CustomLogger
import logging

In [2]:
logger = CustomLogger().info_logger

In [3]:
def asdf():
    for i in range(1):
        logger.info('asdf')
asdf()

[INFO | 2022-02-19 20:35:54,826 | 1375158448.py | asdf] (line 3) asdf :: info logger
